In [1]:
import subprocess
import zipfile
import os
import shutil
import pandas as pd
import numpy as np

//anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
def shape2topojson (src, trg):

    #unzip file
    src_zip = src + "SWE_adm_shp.zip"
    tmp_dir = "tmp/"
    with zipfile.ZipFile(src_zip, "r") as z:
        z.extractall(tmp_dir)
        
    #convert from shape to geojson
    tmp_outfiles = ["counties.json", "municipalities.json"]
    tmp_infiles = ["SWE_adm1.shp", "SWE_adm2.shp"]
    cmd = "ogr2ogr -f GeoJSON "
    
    for i in range(len(tmp_infiles)):
        command = cmd + tmp_dir + tmp_outfiles[i] + " " + tmp_dir + tmp_infiles[i]
        print "Running command " + command
        subprocess.call(command, shell=True)
    
    #convert from geosjon to topojson
    cmd = "topojson -o "
    tmp_outfile = trg + "sweden.json"
    tmp_args = " --id-property HASC_2,HASC_1 -p municipality_name=NAME_2,county_name=NAME_1 -- "
    command = cmd + tmp_outfile + tmp_args + " ".join([tmp_dir + tmp_outfile for tmp_outfile in tmp_outfiles])
    print "Running command " + command
    subprocess.call(command, shell=True)
    
    #create id files
    df = pd.read_csv(tmp_dir + "SWE_adm1.csv", encoding="utf-8", usecols=["NAME_1","HASC_1"])
    df.rename(columns = {"NAME_1": "county", "HASC_1": "domID"}, inplace=True)
    df["county"] = df["county"].str.upper()
    df["domID"] = map(lambda x: "-".join(x.split(".")), df["domID"])
    df.to_csv(trg + "domID-county.csv", index=False, encoding="utf-8")
    
    df = pd.read_csv(tmp_dir + "SWE_adm2.csv", encoding="utf-8", usecols=["NAME_2","HASC_2"])
    df.rename(columns = {"NAME_2": "municipality", "HASC_2": "domID"}, inplace=True)
    df["municipality"] = df["municipality"].str.upper()
    df = df.fillna("SE.VG.ML")
    df = df.set_index("municipality")
    df.loc["KNIVSTA"] = "SE.UP.KN"
    df = df.reset_index()
    df["domID"] = map(lambda x: "-".join(x.split(".")), df["domID"])
    df.to_csv(trg + "domID-municipality.csv", index=False, encoding="utf-8")
    
    del df
    
    #delete tmp dir
    shutil.rmtree(tmp_dir)

In [385]:
if __name__ == "__main__":
    
    #Extract geo data
    shape2topojson ("src/", "output/")

Running command ogr2ogr -f GeoJSON tmp/counties.json tmp/SWE_adm1.shp
Running command ogr2ogr -f GeoJSON tmp/municipalities.json tmp/SWE_adm2.shp
Running command topojson -o output/sweden.json --id-property HASC_2,HASC_1 -p municipality_name=NAME_2,county_name=NAME_1 -- tmp/counties.json tmp/municipalities.json


In [2]:
## paths and filenames

src = "src/"
trg = "output/"

rec = src + "Kommunmottagna+enligt+ersattningsförordningen+2015.xls"
asylum_applications = src + "Inkomna+ansökningar+om+asyl+2015+-+Applications+for+asylum+received+2015.xls"
asylum_granted = src + "Avgjorda+asylärenden+2015+-+Asylum+desicions+2015.xls"
trans = src + "translation.xlsx"
pop_county = src + "BE0101N1_county.xlsx"
pop_municipality = src + "BE0101N1_municipality.xlsx"
ids_county = trg + "domID-county.csv"
ids_muni = trg + "domID-municipality.csv"
income_county = src + "HE0110J8_county.xlsx"
opinion = src + "opinion.csv"

In [3]:
# translation
df_translation = pd.read_excel(trans)
translation = pd.Series(df_translation.new.values,index=df_translation.old).to_dict()

## Asylum

In [10]:
# asylum data
df_applications = pd.read_excel(asylum_applications, sheetname=0, \
                                skiprows=[0,1,2,3,4,5], parse_cols="B:G", skip_footer=4, index_col=0)
df_granted = pd.read_excel(asylum_granted, sheetname=0, \
                           skiprows=[0,1,2,3,4,5,6,7,8], parse_cols="B,D:L", skip_footer=6, index_col=0)

df_merged = pd.concat([df_applications, df_granted], axis=1)
df_merged = df_merged.reset_index()
df_merged["deficit"] = df_merged["Number"] - df_merged["Decisions"]

outfile = trg + "asylum.csv"
df_merged.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_applications
del df_granted
del df_merged

Printing output/asylum.csv


In [15]:
wh = 1812 #working hours per year
h = 247 #handling time
c = 1812 / 247 # cases per year and employee
D = 104075 # deficit at end of 2015

employees = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500]
y = [np.ceil(D/(c*x)) for x in employees]

#data = {"employees": employees, "years_left": y}
#df_deficit = pd.DataFrame(data)

[29.0, 14.0, 9.0, 7.0, 5.0, 4.0, 4.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0]

In [97]:
# applications by country of origin
df_applications_coo = pd.read_excel(asylum_applications, sheetname=1, skiprows=[0,1,2,3,4,5,6,7,8,9], parse_cols="B,D:H")
df_applications_coo = df_applications_coo.fillna(0);
df_applications_coo = df_applications_coo.sort_values("Number", ascending=False);

outfile = trg + "asylum-coo.csv"
df_applications_coo.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_applications_coo

Printing output/asylum-coo.csv


In [116]:
# granted applications by country of origin
df_apps_granted_coo = pd.read_excel(asylum_granted, sheetname=1, skiprows=[0,1,2,3,4,5,6,7,8], parse_cols="B,D:L")


df_apps_granted_coo.iloc[df_apps_granted_coo.count()[0],0] = "Total"
df_apps_granted_coo = df_apps_granted_coo.fillna(0)
df_apps_granted_coo = df_apps_granted_coo.sort_values("of which granted", ascending=False)
df_apps_granted_coo = df_apps_granted_coo.rename(columns={" Citizenship": "Citizenship"})

outfile = trg + "asylum-coo-granted.csv"
df_apps_granted_coo.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_apps_granted_coo

Printing output/asylum-coo-granted.csv


## Arrivals 

In [12]:
# refugees received by reason
df_rec_reason = pd.read_excel(rec, sheetname=5, skiprows=[0,1,2,3,4,5], parse_cols="B,D:P")

# rename columns
df_rec_reason.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "municipality"}, inplace=True)
df_rec_reason.rename(columns = translation, inplace=True)

# fix missing values
df_rec_reason = df_rec_reason.dropna(axis=0,how="all")
df_rec_reason = pd.concat([df_rec_reason.loc[:,df_rec_reason.columns[:2]].fillna(""), \
                         df_rec_reason.loc[:,df_rec_reason.columns[2:]].fillna(0)], axis=1)

outfile = trg + "arrivals-reason.csv"
df_rec_reason.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_rec_reason

Printing output/arrivals-reason.csv


In [13]:
# refugees received by age
df_rec_age = pd.read_excel(rec, sheetname=6, skiprows=[0,1,2,3,4,5], parse_cols="B,D:N")

# rename columns
df_rec_age.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "municipality"}, inplace=True)
df_rec_age.rename(columns = translation, inplace=True)

# fix missing values
df_rec_age = df_rec_age.dropna(axis=0,how="all")
df_rec_age = pd.concat([df_rec_age.loc[:,df_rec_age.columns[:2]].fillna(""), \
                         df_rec_age.loc[:,df_rec_age.columns[2:]].fillna(0)], axis=1)

outfile = trg + "arrivals-age.csv"
df_rec_age.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_rec_age

Printing output/arrivals-age.csv


In [14]:
# refugees received by citizenship
df_rec_coo = pd.read_excel(rec, sheetname=7, skiprows=[0,1,2,3,4,5], parse_cols="B:O")

# rename columns
df_rec_coo.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "citizenship"}, inplace=True)
df_rec_coo.rename(columns = translation, inplace=True)

# fix missing values
df_rec_coo = df_rec_coo.dropna(axis=0,how="all")
df_rec_coo = pd.concat([df_rec_coo.loc[:,df_rec_coo.columns[:2]].fillna(""), \
                         df_rec_coo.loc[:,df_rec_coo.columns[2:]].fillna(0)], axis=1)

outfile = trg + "arrivals-citizenship-full.csv"
df_rec_coo.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

Printing output/arrivals-citizenship-full.csv


In [15]:
# refugees received by citizenship
df_rec_citizenship = pd.read_excel(rec, sheetname=7, skiprows=[0,1,2,3,4,5], parse_cols="B,C,N,O")

# rename columns
df_rec_citizenship.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "citizenship"}, inplace=True)
df_rec_citizenship.rename(columns = translation, inplace=True)

# get unique counties and countries
countries = df_rec_citizenship["citizenship"].dropna().unique()

idx = []
idx.append(np.where(countries == "Totalt")[0][0])
idx.append(np.where(countries == "Delsumma")[0][0])
countries = np.delete(countries, idx)
counties = df_rec_citizenship["county"].unique()

# create new dataframe
header = np.insert(countries[:-1], 0, "county")
df_rec_citizenship = pd.DataFrame(columns = header)
df_rec_citizenship["county"] = counties
df_rec_citizenship = df_rec_citizenship.dropna(axis=0,how="all")
df_rec_citizenship = df_rec_citizenship.set_index("county")

county = ""
country = ""
value = 0
for index, row in df_rec_coo.iterrows():
    if (row["county"] and county != row["county"]):
        county = row["county"]
    
    if (row["citizenship"] != "Delsumma" and row["citizenship"] != "Totalt"):
        country = row["citizenship"]
        value = row["total"]
        df_rec_citizenship.loc[county,country] = value

# fill nans and unset index
df_rec_citizenship = df_rec_citizenship.fillna(0).reset_index()

outfile = trg + "arrivals-citizenship.csv"
df_rec_citizenship.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_rec_coo
del df_rec_citizenship

Printing output/arrivals-citizenship.csv


In [16]:
# refugees received per capita
df_rec = pd.read_excel(rec, sheetname=5, skiprows=[0,1,2,3,4,5], parse_cols="B,D,O")
df_pop = pd.read_excel(pop_county, skiprows=[0,1], skip_footer=40,parse_cols="A:C", \
                       converters={'Unnamed: 0': lambda x: str(x)});
df_pop_muni = pd.read_excel(pop_municipality, skiprows=[0,1], skip_footer=40,parse_cols="A:C", \
                       converters={'Unnamed: 0': lambda x: str(x)});

# rename
df_rec.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "municipality"}, inplace=True)
df_rec.rename(columns = translation, inplace=True)
df_rec.rename(columns = {"total": "received"}, inplace=True)
df_rec = df_rec.dropna(axis=0,how="all")
df_pop.rename(columns = {"Unnamed: 0": "code", "Unnamed: 1": "county", "2015": "total"}, inplace=True)
df_pop_muni.rename(columns = {"Unnamed: 0": "code", "Unnamed: 1": "municipality", "2015": "total"}, inplace=True)

# get only county and total, make county names uppercase
df_rec_muni = df_rec[df_rec.loc[:,"municipality"] != "Delsumma"][["municipality","received"]]
df_rec_muni = df_rec_muni[df_rec_muni.loc[:,"municipality"] != "Totalt"]
df_rec = df_rec[df_rec.loc[:,"municipality"] == "Delsumma"][["county","received"]]
df_pop["county"] = df_pop["county"].str.upper()
df_pop_muni["municipality"] = df_pop_muni["municipality"].str.upper()

In [17]:
df_rec_pc = df_pop.set_index("county")
df_rec_pc_muni = df_pop_muni.set_index("municipality")
df_rec_pc["received"] = 0
df_rec_pc_muni["received"] = 0
county = ""
municipality = ""

for index, row in df_rec.iterrows():
    county = row["county"]
    value = row["received"]
    df_rec_pc.loc[county,"received"] = value
    
for index, row in df_rec_muni.iterrows():
    municipality = row["municipality"]
    value = row["received"]
    if (municipality in df_rec_pc_muni.index):
        df_rec_pc_muni.loc[municipality, "received"] = value
    else:
        df_rec_pc_muni.loc[municipality.replace("-", " "), "received"] = value

# add column for percentage and reorder cols
df_rec_pc = df_rec_pc.reset_index()
df_rec_pc["percent"] = df_rec_pc["received"] / df_rec_pc["total"]
df_rec_pc = df_rec_pc[["code","county","percent","received","total"]]

df_rec_pc_muni = df_rec_pc_muni.reset_index()
df_rec_pc_muni["percent"] = df_rec_pc_muni["received"] / df_rec_pc_muni["total"]
df_rec_pc_muni = df_rec_pc_muni[["code","municipality","percent","received","total"]]

# add domIDs
df_ids = pd.read_csv(ids_county, encoding="utf-8")
df_ids = df_ids.set_index("county")
df_rec_pc = df_rec_pc.set_index("county")
df_rec_pc["domID"] = ""

df_ids_muni = pd.read_csv(ids_muni, encoding="utf-8")
df_ids_muni = df_ids_muni.set_index("municipality")
df_rec_pc_muni = df_rec_pc_muni.set_index("municipality")
df_rec_pc_muni["domID"] = "" 

# counties
for index, row in df_rec_pc.iterrows():
    county = index
    county = " ".join(county.split(" ")[:-1])
    if county[-1] == "S":
        county = county[:-1]
        
    if (county in df_ids.index):
        value = df_ids.loc[county].domID
        df_rec_pc.loc[index, "domID"] = value
        
df_rec_pc = df_rec_pc.reset_index()
df_rec_pc = df_rec_pc.set_index("code")
df_rec_pc.loc["18","domID"] = "SE-OR" # fix örebro
df_rec_pc = df_rec_pc.reset_index()

# municipalities
for index, row in df_rec_pc_muni.iterrows():
    municipality = index
    if (municipality in df_ids_muni.index):
        value = df_ids_muni.loc[municipality].domID
        df_rec_pc_muni.loc[index, "domID"] = value

df_rec_pc_muni = df_rec_pc_muni.reset_index()
df_rec_pc_muni = df_rec_pc_muni.set_index("code")
df_rec_pc_muni.loc["0114", "domID"] = "SE-ST-UV"
df_rec_pc_muni.loc["2023", "domID"] = "SE-KO-MA"
df_rec_pc_muni = df_rec_pc_muni.reset_index()
        
outfile = trg + "arrivals-pc-county.csv"
df_rec_pc.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

outfile = trg + "arrivals-pc-municipality.csv"
df_rec_pc_muni.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

Printing output/arrivals-pc-county.csv
Printing output/arrivals-pc-municipality.csv


In [ ]:
del df_rec
del df_rec_muni
del df_pop
del df_pop_muni
del df_rec_pc
del df_rec_pc_muni
del df_ids
del df_ids_muni

In [31]:
# get median income per month
df_income_county = pd.read_excel(income_county, skiprows=[0,1], skip_footer=31,parse_cols="A,B,E", \
                       converters={'Unnamed: 0': lambda x: str(x)});

df_arrivals_county = pd.read_csv(trg + "arrivals-pc-county.csv")

df_income_county.rename(columns = {"Unnamed: 0": "code", "Unnamed: 1": "county", "2014": "income"}, inplace=True)
df_income_county["income"] = df_income_county["income"] * 1000 / 12
df_income_county["county"] = df_income_county["county"].str.upper()
df_income_county["arrivals"] = df_arrivals_county["received"]

outfile = trg + "median-income-arrivals-county.csv"
df_income_county.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

del df_income_county

Printing output/median-income-arrivals-county.csv


In [24]:
# fix opinion
df_opinion = pd.read_csv(opinion)
df_opinion.rename(columns = translation, inplace=True)

outfile = trg + "opinion.csv"
df_opinion.to_csv(outfile, index=False, encoding="utf-8")
print "Printing " + outfile

Printing output/opinion.csv
